### This notebook generates time-dependent grids of reconstructed points from a reference h3 grid and a plate tectonic model

In [1]:
from pathlib import Path
import h3
import numpy as np
import pandas as pd
import pygplates as pygp
import cartopy.crs as crs        # only needed if plotting
import matplotlib.pyplot as plt  # only needed if plotting

#### User-defined parameters

In [2]:
h3_resolution = 4      # resolution of mesh in integer range: 0-15, where 0 is the coarsest resolution

plate_model = 'MA16'   # plate model  
static_polygons = pygp.FeatureCollection('plate_models/%s/%s_static_polygons.gpmlz' % (plate_model, plate_model))    # static polygon file
rotation_model = 'plate_models/%s/%s_rotation_model.rot' % (plate_model, plate_model)        # rotation file
anchor_plate = 0       # anchor plate ID (sets the effective reference frame) [*** this will generally be set to 0 ***]

start = 1     # start time (in Ma) = youngest reconstruction step (e.g. not t=0)
stop = 410    # stop time (in Ma) = oldest reconstruction step
step = 1      # temporal step size (in Ma)

output_dir = 'rotation_grids/%s' % plate_model
Path(output_dir).mkdir(parents=True, exist_ok=True)
rotation_grid = '%s/resolution_%s.csv' % (output_dir, h3_resolution)        # output file name

plot_grids = 'n'  # plot the starting grid and any redundant points (y/n); this is useful for troubleshooting.

#### Extract each static polygon and assign its corresponding plate ID to all h3 cells which fall within it

In [3]:
df = pd.DataFrame()
for feature in static_polygons:
    plate_id = feature.get_reconstruction_plate_id()
    valid_time = feature.get_valid_time()
    if valid_time[0] > 0 and valid_time[1] <= 0:   # ignore polygons that are only defined at t=0 or those which are not defined at t=0
        geometries = feature.get_all_geometries()

        for geometry in geometries:
            geoJson_polygon = {'type': 'Polygon', 'coordinates': [geometry.to_lat_lon_list()]}
            hexagons = list(h3.polyfill(geoJson_polygon, h3_resolution))
            if hexagons:
                centroids = [h3.h3_to_geo(x) for x in hexagons]
                lats = [x[0] for x in centroids]
                lons = [x[1] for x in centroids]
                pids = [plate_id for x in centroids]
                ages = [valid_time[0] for x in centroids]
                
                new_data = pd.DataFrame(list(zip(hexagons, lons, lats, pids, ages)),
                                      columns =['h3', 'lng', 'lat', 'plt_id', 'max_age'])
                
                updated_df = pd.concat([df, new_data])
                df = updated_df

#### Plot grid (if desired) to ensure that the grid coverage is as expected

In [4]:
if plot_grids == 'y':
    fig = plt.figure(figsize=(15,10))
    ax = fig.add_subplot(1,1,1, projection=crs.PlateCarree())
    #ax = fig.add_subplot(1,1,1, projection=crs.Orthographic(central_latitude=-90, central_longitude=0))
    ax.stock_img()
    ax.coastlines()
    ax.gridlines()
    plt.scatter(x=df.lng, y=df.lat, color="red", s=2, transform=crs.PlateCarree())

    for polygon in static_polygons:
        for geometry in polygon.get_geometries():
            vertices = geometry.to_lat_lon_array()

            plt.plot(vertices[:, 1], vertices[:, 0], transform=crs.Geodetic(),  color='blue', linewidth=0.75)

    #plt.savefig('ex.png')
    plt.show()

#### Check for duplicated h3 cells (that arise from imperfections in the plate polygons or from issues with h3).
Note that h3, being underpinned by a Cartesian reference system, doesn't deal well with the polar regions, and the pole itself in particular. It also is unable to deal with polygons that stretch beyond 180 degrees of longitude (which polygons that cross the pole do by default). This will give rise to unexpected behavior, and so the plate polygon file may need to be adapted by dividing up polygons which cross the pole or which otherwise span more than 180 degrees of longitude. Other problems can arise where there are polygons with holes--the definitions of these inner rings in the .gpml and .shp formats appear not to be recognized by h3, and so again these polygons may need to be modified.

In [5]:
duplicates = df[df.duplicated(['h3'])]  # find all duplicated points
conflicts = df.groupby(['h3']).filter(lambda x: x['plt_id'].nunique() > 1) # find 'problematic' duplicates, where the plate IDs do not match

if len(duplicates) != 0:
    print('%i duplicated polygon coordinates detected, of which %i have conflicting plate IDs' % (len(duplicates), len(conflicts)/2)) 
    
    if plot_grids == 'y':      
        fig = plt.figure(figsize=(15,10))
        ax = fig.add_subplot(1,1,1, projection=crs.PlateCarree())
        #ax = fig.add_subplot(1,1,1, projection=crs.Orthographic(central_latitude=45, central_longitude=-120))
        ax.stock_img()
        ax.coastlines()
        ax.gridlines()
        plt.scatter(x=duplicates.lng, y=duplicates.lat, color="red", s=2, transform=crs.PlateCarree())
        plt.show()
    
    print ('All duplicates with conflicting plate IDs will be discarded; redundant duplicates will be condensed to a single set')
    discard = conflicts['h3'].unique()
    df = df[~df['h3'].isin(discard)]

#### Convert grid points to pygplates reconstructable objects and reconstruct them at each time step

In [6]:
df = df.reset_index(drop=True)
df['pygp_pts'] = df.apply(lambda row: pygp.PointOnSphere((row.lat, row.lng)), axis=1)            # convert lat/lons to pygplates point format and append as new column
df['pygp_feature'] = df.apply(lambda row: pygp.Feature.create_reconstructable_feature(pygp.FeatureType.gpml_unclassified_feature,
                                                                                      row.pygp_pts,
                                                                                      name='reference_point',
                                                                                      valid_time=(row.max_age, pygp.GeoTimeInstant.create_distant_future()),
                                                                                      reconstruction_plate_id=row.plt_id), axis=1)

In [7]:
for t in range(start, stop+step, step):
    #print ('time step', t)  # for high resolutions it can be helpful to routinely print the time-step to keep track of progress
    
    reconstruct = df.loc[(df['plt_id'] != np.nan) & (df['max_age'] >= t)].copy()  # for each timestep select subset of reconstructable points

    reconstructed_geometries = []
    pygp.reconstruct(reconstruct['pygp_feature'], rotation_model, reconstructed_geometries, t, anchor_plate_id = anchor_plate) # reconstruct points 
    
    reconstructed_points = [x.get_reconstructed_geometry().to_lat_lon() for x in reconstructed_geometries]    # extract lat and lon from reconstructed points
    reconstructed_lats = [x[0] for x in reconstructed_points]
    reconstructed_lons = [x[1] for x in reconstructed_points]
    reconstruct[f'lat_{t}'] = reconstructed_lats
    reconstruct[f'lng_{t}'] = reconstructed_lons
    df = pd.concat([df, reconstruct[f'lng_{t}'], reconstruct[f'lat_{t}']], axis=1, join='outer') # append reconstructed lat & lons back to dataframe

#### Clean and format the dataframe + export

In [8]:
df = df.drop(columns=['plt_id', 'max_age', 'pygp_pts', 'pygp_feature'])   # drop columns that we don't need
df.to_csv(rotation_grid, index=False, na_rep='NA', float_format='%.4f')   # set format of NaNs + precision of floats and save